<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/07_14_22_GloVe_XGB_politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RQ1: Glove embeddings for feature Title and Text of the news article. Embeddings fed into the XgBoost ML model to see results. 


In [ ]:
!pip install -U xgboost wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 75 kB/s 
     |████████████████████████████████| 1.8 MB 48.0 MB/s 
     |████████████████████████████████| 146 kB 44.1 MB/s 
     |████████████████████████████████| 181 kB 44.3 MB/s 
     |████████████████████████████████| 63 kB 756 kB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4c11404b1f9b2f99b3c8db338b1d81b20c6e3ee8eb81b13ccac538cbcbf0a045
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import gensim.downloader as api

import xgboost as xgb

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from pathlib import Path

import wandb
from wandb.xgboost import WandbCallback
from IPython.display import clear_output


metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Mounted at /content/drive


In [ ]:
dataset_id = 'politifact'

In [ ]:
df = pd.read_csv(base_dir/f"{dataset_id}_agg_2.csv")
df.text.fillna('', inplace=True)
df.head()

,title,text,tweets,retweets,label,url
0,Actress Emma Stone ‘For the first time in his...,,[],"['1020554564334964741', '1020817527046197248',...",fake,NaN
1,Breaking President Trump makes English the of...,,[],[],fake,NaN
2,Friendly Fire … Charlottesville Car attacker ...,,[],"['3265439004', '3250621593', '3253922920', '32...",fake,NaN
3,If You Are Using This Toothpaste… Throw It Aw...,MightyLiving Blog\n\nHelpful inspiration from ...,"[{'id': 931006425966501893, 'text': 'If You Ar...","['911971426571255810', '1036749614853103616', ...",fake,mightynest.com/learn/research-by-concern/dange...
4,"""Face the Nation"" transcripts, August 26, 2012...","""Face the Nation"" transcripts, August 26, 2012...",[],[],real,https://web.archive.org/web/20120827001956/htt...


In [ ]:
(df.title.str.len() == 0).any()

False

In [ ]:
assert len(df) == len(df.title.unique())

In [ ]:
news = df[['title', 'text', 'label']].copy()

print(news.isna().any())
news.count()

title    False
text     False
label    False
dtype: bool


title    894
text     894
label    894
dtype: int64

Importing libraries to preprocess the title column. 

In [ ]:
import nltk
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
news['text']= news['text'].apply(denoise_text)

In [ ]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.title)

In [ ]:
d_wv = 300
glove = api.load(f'glove-wiki-gigaword-{d_wv}')

[==================================================] 100.0% 376.1/376.1MB downloaded


Creating the embedding matrix to convert features to model-readable format. 

In [ ]:
emb_matrix = np.zeros((max(tokenizer.word_index.values())+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)

In [ ]:
import ast

df['tweets'] = df.tweets.map(ast.literal_eval)

Prepping the features.

In [ ]:
df["num_retweets"] = df.retweets.map(lambda x: len(x.split(',')))
df["log_num_retweets"] = np.log1p(df.num_retweets.to_numpy())
df["num_tweets"] = df.tweets.map(lambda x: len(set([t["id"] for t in x])))
df["log_num_tweets"] = np.log1p(df.num_tweets.to_numpy())

# Training

In [ ]:
WANDB_ENTITY = 'saloniteam'
WANDB_PROJECT = 'nofolds'

The wandb is already logged in a session. Access key to be provided if login is not done. It logs the results into the *nofolds* project in the saloniteam entity (name of account) 

Training the model for titles and full text below respectively. Taking the tokenised text, defining the train parameters, specifiying the name of the run which will be logged in wandb. Once the run is complete, a short summary and the link to wandb is displayed. The wandb tracking can be customised to include different evaluation metrics, and run statistics as well. 

## Titles

In [ ]:
tokenized_titles = tokenizer.texts_to_sequences(news.title.tolist())

X = []
for title in tokenized_titles:
    X.append(emb_matrix[title].mean(0))

X = np.stack(X, axis=0)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 300), (894,))

In [ ]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [ ]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    N = params.get('num_boost_round', 200)
    clf = xgb.train(params, dtrain, num_boost_round=N, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [ ]:
params = {
    "objective":'binary:logistic',
    "seed":124,
    'num_boost_round':200
}

In [ ]:
GROUP = f"{dataset_id}-glove-title-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(train_idx, test_idx, params)
    break

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[18:11:58] WARNING: ../src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.52274	eval-logloss:0.63264
[1]	train-logloss:0.40558	eval-logloss:0.57738
[2]	train-logloss:0.32335	eval-logloss:0.53764
[3]	train-logloss:0.26573	eval-logloss:0.51009
[4]	train-logloss:0.21525	eval-logloss:0.48832
[5]	train-logloss:0.17902	eval-logloss:0.48723
[6]	train-logloss:0.15532	eval-logloss:0.48219
[7]	train-logloss:0.13369	eval-logloss:0.47892
[8]	train-logloss:0.11677	eval-logloss:0.47172
[9]	train-logloss:0.10297	eval-logloss:0.45909
[10]	train-logloss:0.09133	eval-logloss:0.45128
[11]	train-logloss:0.08210	eval-logloss:0.45444
[12]	train-logloss:0.07415	eval-logloss:0.45656
[13]	train-logloss:0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▄▃▃▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
eval/accuracy,0.84358
eval/f1,0.84615
eval/precision,0.89535


## Fulltext

Training for full text + titles now. It can be compared to the performance above to see if there was an improvement or not by adding another feature for real/fake news prediction. 

In [ ]:
news['fulltext'] = news['title'] + '\n' + news['text']

In [ ]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.fulltext)

In [ ]:
emb_matrix = np.zeros((len(tokenizer.word_index)+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)
print(emb_matrix.shape)

(33450, 300)


In [ ]:
tokenized_texts = tokenizer.texts_to_sequences(news.fulltext.tolist())

# maximal number of tokens to use (set to -1 to use full article)
max_len = 128 
X = []
for text_ids in tokenized_texts:
    text_ids = text_ids[:max_len]
    X.append(emb_matrix[text_ids].mean(0))

X = np.stack(X, axis=0)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 300), (894,))

In [ ]:
params = {
    "objective":'binary:logistic',
    "seed":124,
    'num_boost_round':200
}

In [ ]:
GROUP = f"{dataset_id}-me-glove-fulltext-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, group=GROUP, name=f"{GROUP}-{fold_id}", tags=['glove', 'xgb']) as run:
        train(train_idx, test_idx, params)
    break

wandb: Currently logged in as: saloni (saloniteam). Use `wandb login --relogin` to force relogin


[18:12:14] WARNING: ../src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.52696	eval-logloss:0.61568
[1]	train-logloss:0.41723	eval-logloss:0.54795
[2]	train-logloss:0.33532	eval-logloss:0.50581
[3]	train-logloss:0.27260	eval-logloss:0.47416
[4]	train-logloss:0.22333	eval-logloss:0.45915
[5]	train-logloss:0.18656	eval-logloss:0.42178
[6]	train-logloss:0.15786	eval-logloss:0.41525
[7]	train-logloss:0.13376	eval-logloss:0.40534
[8]	train-logloss:0.11567	eval-logloss:0.39289
[9]	train-logloss:0.10092	eval-logloss:0.38169
[10]	train-logloss:0.08871	eval-logloss:0.38449
[11]	train-logloss:0.07824	eval-logloss:0.38074
[12]	train-logloss:0.06986	eval-logloss:0.37336
[13]	train-logloss:0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
eval/accuracy,0.87151
eval/f1,0.87701
eval/precision,0.9011


## + num tweets/retweets

In [ ]:
glove_text_embs = X.copy()

In [ ]:
X = np.concatenate([glove_text_embs, df.num_retweets.to_numpy()[..., None], df.num_tweets.to_numpy()[..., None]], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 302), (894,))

In [ ]:
GROUP = f"{dataset_id}-me-glove-fulltext+tw-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, group=GROUP, name=f"{GROUP}-{fold_id}", tags=['glove', 'xgb']) as run:
        train(train_idx, test_idx, params)
    break

[18:12:28] WARNING: ../src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.49505	eval-logloss:0.57970
[1]	train-logloss:0.37859	eval-logloss:0.51485
[2]	train-logloss:0.30045	eval-logloss:0.47687
[3]	train-logloss:0.24338	eval-logloss:0.44176
[4]	train-logloss:0.19947	eval-logloss:0.41492
[5]	train-logloss:0.16720	eval-logloss:0.38717
[6]	train-logloss:0.13920	eval-logloss:0.36519
[7]	train-logloss:0.11851	eval-logloss:0.34935
[8]	train-logloss:0.10153	eval-logloss:0.32836
[9]	train-logloss:0.08766	eval-logloss:0.32614
[10]	train-logloss:0.07615	eval-logloss:0.31468
[11]	train-logloss:0.06792	eval-logloss:0.30986
[12]	train-logloss:0.06063	eval-logloss:0.30604
[13]	train-logloss:0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▄▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
eval/accuracy,0.88827
eval/f1,0.89583
eval/precision,0.89583


The run can be found in the specified group by name or direct link to wandb. 